In [39]:
using MLJ
using DataFrames
using LinearAlgebra

In [30]:
using Plots

In [3]:
include("/Users/julietnwagwuume-ezeoke/My Drive/CS361_Optim/_fp_cs361/gp_main.jl")

In [36]:
include("/Users/julietnwagwuume-ezeoke/My Drive/CS361_Optim/_fp_cs361/samples.jl")


save_samples (generic function with 2 methods)

In [43]:
include("/Users/julietnwagwuume-ezeoke/My Drive/CS361_Optim/_fp_cs361/pso.jl")

particle_swarm_optimization (generic function with 4 methods)

# Data Conversion

In [33]:
y = convert(Array{Float64,1}, y./10e9);

# convert X from matrix to table 
Xtable = MLJ.table(X./10e9)
println(scitype(Xtable))
schema(Xtable);

Table{AbstractVector{Continuous}}


# GaussianProcessRegressor

In [5]:
# name = "GaussianProcessRegressor"
filter(model) = model.name == "GaussianProcessRegressor" &&
                model.package_name == "ScikitLearn"
models(filter)[1];

In [6]:
GPModel = @load GaussianProcessRegressor pkg="ScikitLearn" verbosity=0



MLJScikitLearnInterface.GaussianProcessRegressor

In [7]:
gpmodel = GPModel()

GaussianProcessRegressor(
    kernel = nothing,
    alpha = 1.0e-10,
    optimizer = "fmin_l_bfgs_b",
    n_restarts_optimizer = 0,
    normalize_y = false,
    copy_X_train = true,
    random_state = nothing) @033

In [8]:
r = range(gpmodel, :alpha, lower=10e-12, upper=10e-5, scale=:log);

In [9]:
self_tuning_gpmodel = TunedModel(model=gpmodel,
							  resampling=CV(nfolds=3),
							  tuning=Grid(resolution=10),
							  range=r,
							  measure=rms);

In [27]:
mach = machine(self_tuning_gpmodel, Xtable, y);
fit!(mach, verbosity=0)

Machine{DeterministicTunedModel{Grid,…},…} @212 trained 1 time; caches data
  args: 
    1:	Source @403 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @996 ⏎ `AbstractVector{Continuous}`


In [28]:
fitted_params(mach).best_model

GaussianProcessRegressor(
    kernel = nothing,
    alpha = 5.99484250318941e-11,
    optimizer = "fmin_l_bfgs_b",
    n_restarts_optimizer = 0,
    normalize_y = false,
    copy_X_train = true,
    random_state = nothing) @141

In [29]:
# did not impact rmse much :( 
entry = report(mach).best_history_entry

(model = GaussianProcessRegressor @141,
 measure = [RootMeanSquaredError @409],
 measurement = [1.3311155389268386e-10],
 per_fold = [[1.2547715501911238e-10, 1.258646359890336e-10, 1.4686604207519295e-10]],)

In [55]:
function fpred(X)
    return MLJ.predict(mach, X)
end


fpred (generic function with 1 method)

# Sensitivity Analysis

In [32]:
# # generate grid of samples 
# function samples_full_factorial(a,b,m)
#     ranges = [range]
# end

In [ ]:
# forward / backward differentiation to calculate the indices 

# PSO

In [ ]:
# generate x 
positions = get_filling_set_halton(50, 61); # TODO turn arrays to vectors

In [48]:
positions[1, :][1]

61-element Vector{Float64}:
 0.5
 0.3333333333333333
 0.2
 0.14285714285714285
 0.09090909090909091
 0.07692307692307693
 0.058823529411764705
 0.05263157894736842
 0.043478260869565216
 0.034482758620689655
 ⋮
 0.004149377593360996
 0.00398406374501992
 0.0038910505836575876
 0.0038022813688212928
 0.0037174721189591076
 0.0036900369003690036
 0.0036101083032490976
 0.0035587188612099642
 0.0035335689045936395

In [42]:
# generate v 
# I matrix, 61 * 61, then take first 50 
velocities = Matrix(1.0I, 61, 61)[1:50, :];  # Identity matrix of Float64 type

In [47]:
velocities[1, :]

61-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [51]:
size(positions, 1)

50

In [53]:
# create population 
population = []
for i in 1:size(positions, 1)
    x = positions[i, :][1]
    v = velocities[i, :]
    push!(population, Particle(x, v, x))
end

population;

In [ ]:
particle_swarm_optimization(fpred, population, 100, w=1, c1=1, c2=1)